# Test 3

In the past two tests, we have built the basic multi-agent workflow, and finalized the decisions of framework(Langraph) and LLM(Gemini-2.5-flash) usage. Test 3 mainly focuses on:

- Add the RAG workflow for Agent B (Generate Questions and Teacher Prompts) to retrieve knowledge from the course library
- Refine the prompts based on the contexts engineering and MCP.

## RAG integration

requirements.txt
langgraph==0.2.34
langchain-core==0.3.15
langchain-google-genai==2.0.5
python-dotenv==1.0.1
pydantic==2.9.2

In [ ]:
from langgraph.graph import StateGraph
from langchain_core.messages import HumanMessage
from langchain_core.runnables import Runnable
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
from pydantic import BaseModel
import os
import json
from pathlib import Path
from typing import List, Dict, Any, Optional
import logging
import re

# Setup loggin
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()

# Initialize LLM and embeddings
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.3)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Define the state schema
class SharedInput(BaseModel):
    input: str
    course_id: str = "default"
    extracted_info: str = None
    questions: str = None
    prompts: str = None
    evaluation: str = None
    relevant_context: str = None

class SimpleCourseLibrary:
    """Simplified course library without vector store dependencies"""
    
    def __init__(self):
        self.course_libraries = {}
        self.library_path = Path("course_libraries")
        self.library_path.mkdir(exist_ok=True)
    
    def add_course_document(self, course_id: str, title: str, content: str, 
                          author: str = "", doc_type: str = "text"):
        """Add a document to course library"""
        if course_id not in self.course_libraries:
            self.course_libraries[course_id] = []
        
        document = {
            "id": len(self.course_libraries[course_id]) + 1,
            "title": title,
            "content": content,
            "author": author,
            "type": doc_type
        }
        
        self.course_libraries[course_id].append(document)
        self._save_course_library(course_id)
        return document
    
    def _save_course_library(self, course_id: str):
        """Save course library to file"""
        file_path = self.library_path / f"{course_id}.json"
        with open(file_path, 'w') as f:
            json.dump(self.course_libraries[course_id], f, indent=2)
    
    def load_course_library(self, course_id: str):
        """Load course library from file"""
        file_path = self.library_path / f"{course_id}.json"
        if file_path.exists():
            with open(file_path, 'r') as f:
                self.course_libraries[course_id] = json.load(f)
        return self.course_libraries.get(course_id, [])
    
    def search_documents(self, course_id: str, query: str, max_results: int = 3) -> List[Dict]:
        """Simple text-based search without vector embeddings"""
        documents = self.load_course_library(course_id)
        query_words = set(query.lower().split())
        
        scored_docs = []
        for doc in documents:
            # Simple scoring based on keyword matches
            doc_text = f"{doc['title']} {doc['content']}".lower()
            doc_words = set(doc_text.split())
            
            # Calculate overlap score
            overlap = len(query_words.intersection(doc_words))
            if overlap > 0:
                scored_docs.append((overlap, doc))
        
        # Sort by score and return top results
        scored_docs.sort(key=lambda x: x[0], reverse=True)
        return [doc for score, doc in scored_docs[:max_results]]

# Initialize course library
course_library = SimpleCourseLibrary()

# Agent A: Extract Information and Find Relevant Context
class ExtractorAgent(Runnable):
    def invoke(self, state, config=None):
        # Extract main ideas
        extract_prompt = f"Extract the main ideas and keywords from this text:\n\n{state.input}"
        extract_response = llm.invoke([HumanMessage(content=extract_prompt)])
        extracted_info = extract_response.content
        
        # Search for relevant context using simple text matching
        relevant_context = ""
        try:
            relevant_docs = course_library.search_documents(state.course_id, extracted_info)
            
            if relevant_docs:
                context_pieces = []
                for doc in relevant_docs:
                    context_pieces.append(f"Title: {doc['title']}\nAuthor: {doc['author']}\nContent: {doc['content'][:500]}...")
                
                relevant_context = "\n\n---\n\n".join(context_pieces)
            else:
                relevant_context = "No relevant course materials found."
                
        except Exception as e:
            logger.error(f"Error retrieving context: {str(e)}")
            relevant_context = "Error retrieving course materials."
        
        return {
            "extracted_info": extracted_info,
            "relevant_context": relevant_context
        }

# Agent B: Generate Questions and Teacher Prompts
class ScaffoldPromptAgent(Runnable):
    def invoke(self, state, config=None):
        context_section = ""
        if state.relevant_context and state.relevant_context != "No relevant course materials found.":
            context_section = f"\n\nRelevant Course Materials:\n{state.relevant_context}\n"
        
        prompt = (
            "Based on the following information and any relevant course materials, create:\n\n"
            "1. Generate 4 comprehension questions aligned with the Reading Apprenticeship (RA) framework:\n"
            "   - Social: Promotes discussion or collaborative thinking\n"
            "   - Personal: Invites self-reflection or personal connection\n"
            "   - Cognitive: Encourages metacognition or strategy use\n"
            "   - Knowledge-Building: Deepens understanding of concepts\n\n"
            "2. Create a short teacher prompt (1–2 sentences) for each question to guide facilitation.\n\n"
            "Format your response exactly as follows:\n"
            "Questions:\n"
            "1. [Social question]\n"
            "2. [Personal question]\n"
            "3. [Cognitive question]\n"
            "4. [Knowledge-Building question]\n\n"
            "Prompts:\n"
            "1. [Teacher prompt for question 1]\n"
            "2. [Teacher prompt for question 2]\n"
            "3. [Teacher prompt for question 3]\n"
            "4. [Teacher prompt for question 4]\n\n"
            f"Primary Information:\n{state.extracted_info}\n"
            f"{context_section}"
        )
        
        response = llm.invoke([HumanMessage(content=prompt)])
        output = response.content
        
        try:
            # Split output into questions and prompts
            if "Prompts:" in output:
                questions_part = output.split("Prompts:")[0].replace("Questions:", "").strip()
                prompts_part = output.split("Prompts:")[1].strip()
            else:
                questions_part = output
                prompts_part = "Prompts not properly formatted."
        except Exception as e:
            logger.error(f"Error parsing output: {str(e)}")
            questions_part = output
            prompts_part = "Error parsing prompts."
        
        return {
            "questions": questions_part,
            "prompts": prompts_part
        }

# Agent C: Quality Check
class QualityAgent(Runnable):
    def invoke(self, state, config=None):
        prompt = (
            "Evaluate the following Reading Apprenticeship questions and teacher prompts:\n\n"
            "For each question, provide:\n"
            "- Which RA dimension it aligns with (Social, Personal, Cognitive, or Knowledge-Building)\n"
            "- A brief justification (1-2 sentences)\n"
            "- Assessment of clarity and educational value\n\n"
            "For each teacher prompt, evaluate:\n"
            "- How well it guides teacher facilitation\n"
            "- Whether it promotes meaningful student engagement\n\n"
            f"Questions:\n{state.questions}\n\n"
            f"Prompts:\n{state.prompts}\n\n"
            "Provide a structured evaluation with specific feedback for improvement."
        )
        
        response = llm.invoke([HumanMessage(content=prompt)])
        return {"evaluation": response.content}

# Create the LangGraph workflow
def create_workflow():
    workflow = StateGraph(SharedInput)
    
    # Add nodes (agents)
    workflow.add_node("extractor", ExtractorAgent())
    workflow.add_node("scaffold_prompt", ScaffoldPromptAgent())
    workflow.add_node("quality", QualityAgent())
    
    # Define graph edges
    workflow.set_entry_point("extractor")
    workflow.add_edge("extractor", "scaffold_prompt")
    workflow.add_edge("scaffold_prompt", "quality")
    workflow.set_finish_point("quality")
    
    # Compile the graph
    return workflow.compile()

# Initialize the workflow
workflow_app = create_workflow()

def setup_example_course():
    """Set up an example course with sample documents"""
    course_id = "biology-101"
    
    # Add sample documents
    course_library.add_course_document(
        course_id=course_id,
        title="Introduction to Photosynthesis",
        content="""
        Photosynthesis is the process by which plants, algae, and certain bacteria convert light energy, 
        usually from the sun, into chemical energy stored in glucose molecules. This process is fundamental 
        to life on Earth as it provides the primary source of energy for most ecosystems.

        The process occurs in two main stages: the light-dependent reactions and the light-independent 
        reactions (Calvin cycle). During the light-dependent reactions, chlorophyll and other pigments 
        absorb light energy, which is used to split water molecules and produce ATP and NADPH. 

        In the Calvin cycle, carbon dioxide from the atmosphere is fixed into organic molecules using 
        the ATP and NADPH produced in the first stage. The overall equation for photosynthesis is:
        6CO2 + 6H2O + light energy → C6H12O6 + 6O2

        Photosynthesis not only provides energy for plants but also produces oxygen as a byproduct, 
        which is essential for the survival of most living organisms on Earth.
        """,
        author="Dr. Emily Smith",
        doc_type="textbook_chapter"
    )
    
    course_library.add_course_document(
        course_id=course_id,
        title="Cellular Respiration and Energy Production",
        content="""
        Cellular respiration is the process by which cells break down glucose and other organic molecules 
        to release energy in the form of ATP (adenosine triphosphate). This process is essentially the 
        opposite of photosynthesis and occurs in all living organisms.

        Cellular respiration consists of three main stages:
        1. Glycolysis: Glucose is broken down into pyruvate in the cytoplasm
        2. Krebs Cycle (Citric Acid Cycle): Pyruvate is further broken down in the mitochondria
        3. Electron Transport Chain: The final stage where most ATP is produced

        The overall equation for cellular respiration is:
        C6H12O6 + 6O2 → 6CO2 + 6H2O + ATP

        This process is crucial for providing energy for all cellular activities, from muscle contraction 
        to protein synthesis. Understanding the relationship between photosynthesis and cellular respiration 
        helps students grasp the fundamental energy cycles that sustain life.
        """,
        author="Dr. Michael Johnson",
        doc_type="lecture_notes"
    )
    
    course_library.add_course_document(
        course_id=course_id,
        title="Plant Cell Structure and Function",
        content="""
        Plant cells have several unique structures that distinguish them from animal cells. The most 
        notable features include the cell wall, chloroplasts, and large central vacuole.

        The cell wall provides structural support and protection, made primarily of cellulose. 
        Chloroplasts are the organelles where photosynthesis occurs, containing the green pigment 
        chlorophyll that captures light energy.

        The central vacuole serves multiple functions including maintaining turgor pressure, 
        storing water and nutrients, and providing structural support. These adaptations allow 
        plants to perform photosynthesis efficiently and maintain their structure without a skeletal system.

        Understanding plant cell structure is essential for comprehending how photosynthesis works 
        at the cellular level and how plants have evolved to capture and convert solar energy.
        """,
        author="Dr. Sarah Wilson",
        doc_type="study_guide"
    )
    
    print(f"Example course '{course_id}' set up with {len(course_library.course_libraries[course_id])} documents")
    return course_id

def run_example():
    """Run an example of the system"""
    # Set up example course
    course_id = setup_example_course()
    
    # Example input text
    input_text = """
    Photosynthesis is a fundamental biological process used by green plants, algae, and some bacteria to convert 
    light energy, usually from the sun, into chemical energy stored in glucose. During this process, organisms absorb 
    carbon dioxide from the air and water from the soil. Using sunlight captured by chlorophyll—the green pigment in 
    plant cells—they convert these raw materials into glucose, a type of sugar that serves as food, and release oxygen 
    as a byproduct. This process not only sustains the plant's growth and energy needs but also produces the oxygen essential 
    for the survival of most living organisms on Earth.
    """
    
    print("\n" + "="*80)
    print("READING APPRENTICESHIP QUESTION GENERATOR")
    print("="*80)
    print(f"\nCourse ID: {course_id}")
    print(f"Input Text: {input_text[:100]}...")
    
    # Create input state
    input_state = SharedInput(
        input=input_text,
        course_id=course_id
    )
    
    # Run the workflow
    print("\nRunning workflow...")
    result = workflow_app.invoke(input_state)
    
    # Display results
    print("\n" + "-"*60)
    print("STEP 1: EXTRACTED INFORMATION")
    print("-"*60)
    print(result.get("extracted_info", "No extracted info"))
    
    print("\n" + "-"*60)
    print("STEP 2: RELEVANT COURSE CONTEXT")
    print("-"*60)
    print(result.get("relevant_context", "No context found"))
    
    print("\n" + "-"*60)
    print("STEP 3: GENERATED QUESTIONS")
    print("-"*60)
    print(result.get("questions", "No questions generated"))
    
    print("\n" + "-"*60)
    print("STEP 4: TEACHER PROMPTS")
    print("-"*60)
    print(result.get("prompts", "No prompts generated"))
    
    print("\n" + "-"*60)
    print("STEP 5: QUALITY EVALUATION")
    print("-"*60)
    print(result.get("evaluation", "No evaluation"))
    
    print("\n" + "="*80)
    print("WORKFLOW COMPLETED SUCCESSFULLY!")
    print("="*80)

if __name__ == "__main__":
    # Check if required environment variables are set
    if not os.getenv("GOOGLE_API_KEY"):
        print("Warning: GOOGLE_API_KEY not found in environment variables.")
        print("Please set your Google API key in a .env file or environment variable.")
        print("Example .env file:")
        print("GOOGLE_API_KEY=your_google_api_key_here")
    else:
        run_example()

## Result 

Example course 'biology-101' set up with 3 documents

================================================================================
READING APPRENTICESHIP QUESTION GENERATOR
================================================================================

Course ID: biology-101
Input Text: 
    Photosynthesis is a fundamental biological process used by green plants, algae, and some bacter...

Running workflow...

------------------------------------------------------------
STEP 1: EXTRACTED INFORMATION
------------------------------------------------------------
Here are the main ideas and keywords from the text:

**Main Ideas:**

*   Photosynthesis is a fundamental biological process that converts light energy (from the sun) into chemical energy, stored as glucose.
*   It is performed by green plants, algae, and some bacteria.
*   The process uses carbon dioxide and water as raw materials, with sunlight captured by chlorophyll.
*   It produces glucose (sugar/food) for the organism and releases oxygen as a byproduct.
*   Photosynthesis is crucial for the plant's growth and energy, and it produces the oxygen necessary for the survival of most life on Earth.

**Keywords:**

*   Photosynthesis
*   Biological process
*   Light energy
*   Chemical energy
*   Glucose (sugar, food)
*   Green plants
*   Algae
*   Bacteria
*   Carbon dioxide
*   Water
*   Sunlight
*   Chlorophyll (green pigment)
*   Oxygen (byproduct)
*   Plant growth
*   Survival (of organisms)

------------------------------------------------------------
STEP 2: RELEVANT COURSE CONTEXT
------------------------------------------------------------
Title: Introduction to Photosynthesis
Author: Dr. Emily Smith
Content: 
        Photosynthesis is the process by which plants, algae, and certain bacteria convert light energy, 
        usually from the sun, into chemical energy stored in glucose molecules. This process is fundamental 
        to life on Earth as it provides the primary source of energy for most ecosystems.

        The process occurs in two main stages: the light-dependent reactions and the light-independent 
        reactions (Calvin cycle). During the light-dependent reactions, chlorophyll and o...

---

Title: Cellular Respiration and Energy Production
Author: Dr. Michael Johnson
Content: 
        Cellular respiration is the process by which cells break down glucose and other organic molecules 
        to release energy in the form of ATP (adenosine triphosphate). This process is essentially the 
        opposite of photosynthesis and occurs in all living organisms.

        Cellular respiration consists of three main stages:
        1. Glycolysis: Glucose is broken down into pyruvate in the cytoplasm
        2. Krebs Cycle (Citric Acid Cycle): Pyruvate is further broken down in ...

---

Title: Plant Cell Structure and Function
Author: Dr. Sarah Wilson
Content: 
        Plant cells have several unique structures that distinguish them from animal cells. The most 
        notable features include the cell wall, chloroplasts, and large central vacuole.

        The cell wall provides structural support and protection, made primarily of cellulose. 
        Chloroplasts are the organelles where photosynthesis occurs, containing the green pigment 
        chlorophyll that captures light energy.

        The central vacuole serves multiple functions including...

------------------------------------------------------------
STEP 3: GENERATED QUESTIONS
------------------------------------------------------------
1.  **Social:** Imagine you are working in a small group. How would you collectively explain the importance of photosynthesis to someone who knows nothing about biology, ensuring they understand its role for both plants and the survival of most life on Earth?
2.  **Personal:** Reflect on your own experiences. How does understanding the process of photosynthesis, particularly its role in producing oxygen and food, make you think differently about the plants and trees in your everyday environment?
3.  **Cognitive:** As you read about photosynthesis, what was one specific part or concept that you found challenging to grasp initially? What mental strategy or approach did you use, or could you use, to help clarify your understanding?
4.  **Knowledge-Building:** The course materials state that cellular respiration is "essentially the opposite" of photosynthesis. Based on the information provided, explain what this statement means by comparing the key inputs and outputs of both processes.

------------------------------------------------------------
STEP 4: TEACHER PROMPTS
------------------------------------------------------------
1.  **Social:** "Work with your group to brainstorm the most crucial points. How can you make this complex process clear and relevant to a general audience?"
2.  **Personal:** "Take a moment to connect this scientific concept to your personal world. How does this new knowledge shift your perspective?"
3.  **Cognitive:** "Think about your reading process. What specific steps did you take when you encountered a difficult idea, or what could you try next time?"
4.  **Knowledge-Building:** "Look closely at the inputs and outputs for both processes. How do they mirror each other, and what does that tell us about their relationship?"

------------------------------------------------------------
STEP 5: QUALITY EVALUATION
------------------------------------------------------------
This is a well-designed set of Reading Apprenticeship questions and teacher prompts, demonstrating a strong understanding of each RA dimension.

---

## Evaluation of Reading Apprenticeship Questions

### 1. Social Question

*   **RA Dimension:** Social
*   **Justification:** The question explicitly requires students to "work in a small group" and "collectively explain," emphasizing collaborative learning and the shared construction of understanding for an audience.
*   **Assessment of Clarity and Educational Value:**
    *   **Clarity:** Excellent. The task is clearly defined, including the audience ("someone who knows nothing about biology") and the core concept to be explained.
    *   **Educational Value:** High. This question promotes deep understanding by requiring students to synthesize complex information, anticipate audience needs, and articulate their knowledge collaboratively. It fosters communication skills, active listening, and the ability to negotiate meaning within a group, all crucial for social learning.

### 2. Personal Question

*   **RA Dimension:** Personal
*   **Justification:** It prompts students to "reflect on your own experiences" and consider how new knowledge "make you think differently" about their "everyday environment," directly connecting the content to their individual lives and perspectives.
*   **Assessment of Clarity and Educational Value:**
    *   **Clarity:** Excellent. The prompt is direct and invites personal introspection.
    *   **Educational Value:** High. This question is vital for fostering intrinsic motivation and making learning relevant. By connecting scientific concepts to personal experiences, students are more likely to internalize the information, see its value, and develop a deeper appreciation for the subject matter. It encourages a shift in perspective, which is a key outcome of meaningful learning.

### 3. Cognitive Question

*   **RA Dimension:** Cognitive
*   **Justification:** The question asks students to identify a "challenging" concept and articulate a "mental strategy or approach" used to overcome that difficulty, directly focusing on metacognition and comprehension strategies.
*   **Assessment of Clarity and Educational Value:**
    *   **Clarity:** Excellent. It clearly asks for both the problem and the solution (strategy).
    *   **Educational Value:** Very High. This question is fundamental to developing skilled readers and learners. It explicitly encourages metacognition – thinking about one's own thinking process. By prompting students to articulate their strategies, it helps them become more aware of effective learning techniques, empowering them to apply these strategies independently in future learning situations.

### 4. Knowledge-Building Question

*   **RA Dimension:** Knowledge-Building
*   **Justification:** It requires students to interpret a statement from the "course materials," compare "key inputs and outputs" of two processes, and explain the relationship between them, demonstrating a synthesis and deeper understanding of content.
*   **Assessment of Clarity and Educational Value:**
    *   **Clarity:** Excellent. The question clearly outlines the task: explain the "opposite" statement by comparing specific elements (inputs/outputs).
    *   **Educational Value:** High. This question moves beyond simple recall to higher-order thinking skills such as analysis, comparison, and synthesis. It assesses students' ability to connect related concepts, build a coherent understanding of a system, and articulate complex relationships, which are core to knowledge construction.

---

## Evaluation of Reading Apprenticeship Teacher Prompts

### 1. Social Prompt

*   **How well it guides teacher facilitation:** Excellent. The prompt provides clear, actionable instructions for the teacher: "Work with your group to brainstorm the most crucial points. How can you make this complex process clear and relevant to a general audience?" It directs the teacher to facilitate group work, focus on key concepts, and encourage audience awareness.
*   **Whether it promotes meaningful student engagement:** Yes, highly. By emphasizing collaboration ("Work with your group") and a real-world application ("make this complex process clear and relevant to a general audience"), it encourages active participation, negotiation of ideas, and a sense of purpose beyond just answering a question.

### 2. Personal Prompt

*   **How well it guides teacher facilitation:** Excellent. The prompt "Take a moment to connect this scientific concept to your personal world. How does this new knowledge shift your perspective?" clearly guides the teacher to encourage individual reflection and personal relevance. It's concise and effective.
*   **Whether it promotes meaningful student engagement:** Yes, highly. It invites students to internalize the learning by linking it to their own lives and experiences. This personal connection makes the content more memorable and meaningful, fostering deeper engagement than purely academic questions.

### 3. Cognitive Prompt

*   **How well it guides teacher facilitation:** Excellent. The prompt "Think about your reading process. What specific steps did you take when you encountered a difficult idea, or what could you try next time?" directly guides the teacher to prompt metacognitive reflection. It encourages a growth mindset by asking about both past strategies and future possibilities.
*   **Whether it promotes meaningful student engagement:** Yes, highly. It empowers students by making them aware of their own learning processes and strategies. This fosters self-efficacy and encourages students to take ownership of their learning, leading to more active and strategic engagement with texts.

### 4. Knowledge-Building Prompt

*   **How well it guides teacher facilitation:** Excellent. The prompt "Look closely at the inputs and outputs for both processes. How do they mirror each other, and what does that tell us about their relationship?" provides specific guidance for the teacher to direct students towards a detailed comparison and analysis of the relationship between concepts.
*   **Whether it promotes meaningful student engagement:** Yes, highly. It pushes students beyond surface-level understanding to analyze, synthesize, and infer relationships. This type of analytical thinking is highly engaging as it challenges students to construct deeper meaning rather than simply recalling facts.

---

## Overall Feedback and Suggestions for Improvement

**Overall Feedback:**
This is an exceptionally strong set of Reading Apprenticeship questions and teacher prompts. Each item is clearly aligned with its stated RA dimension, demonstrates high clarity, and possesses significant educational value. The teacher prompts effectively guide facilitation towards meaningful student engagement, ensuring that the questions are not just asked, but truly explored.

**Suggestions for Improvement:**
While the current set is excellent, here are a few minor considerations for further enhancement, mostly focusing on potential extensions or variations:

1.  **For Social Questions/Prompts:**
    *   **Consider a "product" or "audience" beyond just explanation:** While explaining is great, sometimes a social task can involve creating a shared artifact (e.g., a collaborative diagram, a group presentation slide, a shared summary document). This could add another layer of engagement and tangible outcome.
    *   **Example extension:** "As a group, create a simple infographic or a 60-second video script that explains photosynthesis to a 5th grader, highlighting its importance for life on Earth."

2.  **For Personal Questions/Prompts:**
    *   **Vary the "personal connection":** While "everyday environment" is good, personal connections can also be to future aspirations (e.g., "How might understanding photosynthesis be relevant if you pursued a career in agriculture or environmental science?"), or ethical considerations (e.g., "How does this understanding influence your views on deforestation or climate change?").
    *   **Example extension:** "Beyond your immediate environment, how does understanding photosynthesis connect to a global issue or challenge that you care about?"

3.  **For Cognitive Questions/Prompts:**
    *   **Encourage sharing of strategies:** The current question focuses on individual reflection, which is crucial. However, a follow-up or a slight modification could encourage students to share their strategies with peers, fostering a community of learners who can learn from each other's metacognitive approaches.
    *   **Example extension:** "Share the strategy you used with a partner. Did they use a similar approach, or did they have a different one that might also be helpful?"

4.  **For Knowledge-Building Questions/Prompts:**
    *   **Introduce a "discrepant event" or "new information":** To push knowledge-building even further, sometimes introducing a piece of information that seems to contradict or complicate the initial understanding can be powerful. This forces students to reconcile new information with existing knowledge, strengthening their conceptual framework.
    *   **Example extension:** "Given what you know about photosynthesis and cellular respiration, how would you explain why plants still need oxygen, even though they produce it?" (This challenges the simple "opposite" idea and requires deeper synthesis).

In summary, these questions and prompts are highly effective and demonstrate a sophisticated grasp of the Reading Apprenticeship framework. The suggestions above are minor refinements that could be considered for future iterations or to add variety to the learning experience.

================================================================================
WORKFLOW COMPLETED SUCCESSFULLY!
================================================================================